In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torchvision

import matplotlib.pyplot as plt

TRAIN = False

In [ ]:
if TRAIN:
    train = pd.read_csv('../input/Kannada-MNIST/train.csv').to_numpy()

    X_train, y_train = train[:, 1:], train[:, 0]
    X_train, y_train = torch.tensor(X_train).to(torch.float), torch.tensor(y_train)
    dset_train = data.TensorDataset(X_train.reshape(-1, 1, 28, 28), y_train)

    factor = 0.9
    train_len = int(len(dset_train) * factor)
    valid_len = len(dset_train)  - train_len

    dset_train, dset_valid = data.random_split(dset_train, [train_len, valid_len])

    batch_size = 16

    dloader_train = data.DataLoader(dset_train, batch_size=batch_size, shuffle=True)
    dloader_valid = data.DataLoader(dset_valid, batch_size=batch_size, shuffle=True)

In [ ]:
if TRAIN:
    x, y = next(iter(dloader_train))
    plt.imshow(torchvision.utils.make_grid(x).numpy().transpose(1, 2, 0))
    print(y)

In [ ]:
class Model(nn.Module):
    '''
    Convolutional part
    conv -> relu -> pool -> dropout
    FC part
    fc -> relu'''
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 3,   padding=1)   
        self.conv2 = nn.Conv2d(16, 32, 3,  padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3,  padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2)
    
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.dropout = nn.Dropout(0.25)
        
        self.fc1 = nn.Linear(128 * 7**2, 128)
        self.fc2 = nn.Linear(128, 10)

        
    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)), 0.1)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.1)
        x = self.pool(x)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.1)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.1)
        x = self.pool(x)
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.dropout(self.fc1(x)))
        x = self.fc2(x)
        return x

In [ ]:
if TRAIN:
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = Model().to(device)
    ℓ_fn = nn.CrossEntropyLoss()
    lr = 0.001
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    epochs = 20
    for i_epoch in range(1, epochs+1):
        for x, y in dloader_train:
            x, y = x.to(device), y.to(device)
            ŷ = model(x)
            ℓ = ℓ_fn(ŷ, y)
            optimizer.zero_grad()
            ℓ.backward()
            optimizer.step()
        model.eval()
        loss_val = []
        loss_acc = []
        for x, y in dloader_train:
            x, y = x.to(device), y.to(device)
            ŷ = model(x)
            ℓ = ℓ_fn(ŷ, y)
            loss_val.append(ℓ.item())
            loss_acc.append((y == ŷ.argmax(dim=1)).to(torch.float).mean())
        model.train()
        print(f'Epoch {i_epoch} loss: {sum(loss_val)/len(loss_val):.3f}, aacuracy: {sum(loss_acc)/len(loss_acc):.4f}')
    torch.save(model.state_dict(), 'model.pt')

In [ ]:
# test dataset
test = pd.read_csv('../input/Kannada-MNIST/test.csv').to_numpy()
X_test, X_id = test[:, 1:], test[:, 0]
X_test, X_id = torch.tensor(X_test).to(torch.float), torch.tensor(X_id)
dset_test = data.TensorDataset(X_test.reshape(-1, 1, 28, 28), X_id)
dloader_test = data.DataLoader(dset_test, batch_size=len(dset_test), shuffle=False)

In [ ]:
model = Model()
model.cpu()
model.load_state_dict(torch.load('../input/trained-model-kannada/model.pt',
                                 map_location=torch.device('cpu')))

model.eval()
x, x_id = next(iter(dloader_test))
ŷ = model(x)
model.train()

submission  = pd.DataFrame({'id': x_id, 'label': ŷ.argmax(dim=1)})
submission.to_csv('submission.csv',index=False)